# 🛒 Superstore Sales Analysis using SQL in Jupyter

Welcome to a real-world data analysis project using the Superstore sales dataset.  
In this notebook, we use **Python (Pandas + SQLite)** to run SQL queries on real e-commerce sales data.

This project demonstrates skills in:

- Data Cleaning & Preparation
- SQL Query Writing
- Exploratory Sales Analysis
- Customer Segmentation
- Date-based Aggregations (Time Series)
- Analytical Thinking using SQL

The dataset used here is inspired by a common business scenario: **analyzing retail performance across regions, customers, and product categories**.

Let's dive in! 🚀


In [1]:
import pandas as pd
import sqlite3

## 📂 Step 1: Load Dataset and Preview

We load the Superstore dataset from a CSV file and preview the structure.

In [4]:
df = pd.read_csv("Superstore Sales Data.csv")
df.head()

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,State,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales
0,1,CA-2017-152156,08/11/2017,11/11/2017,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420.0,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600
1,2,CA-2017-152156,08/11/2017,11/11/2017,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420.0,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400
2,3,CA-2017-138688,12/06/2017,16/06/2017,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,California,90036.0,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200
3,4,US-2016-108966,11/10/2016,18/10/2016,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,Florida,33311.0,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775
4,5,US-2016-108966,11/10/2016,18/10/2016,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,Florida,33311.0,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680


## 🧹 Step 2: Convert Date Columns

SQLite requires dates in `YYYY-MM-DD` format, so we:
- Parse the `Order Date` and `Ship Date` columns using Pandas
- Convert them to standardized strings for SQLite compatibility

In [18]:
# Convert Order Date and Ship Date from DD/MM/YYYY to proper datetime
df["Order Date"] = pd.to_datetime(df["Order Date"], dayfirst=True)
df["Ship Date"] = pd.to_datetime(df["Ship Date"], dayfirst=True)

In [19]:
# Format to 'YYYY-MM-DD' strings for SQLite compatibility
df["Order Date"] = df["Order Date"].dt.strftime('%Y-%m-%d')
df["Ship Date"] = df["Ship Date"].dt.strftime('%Y-%m-%d')

## 🗃️ Step 3: Load Data into SQLite

We now insert the cleaned dataset into a local SQLite database, creating a table named `orders`.

In [20]:
# Connect or reconnect to the SQLite database
conn = sqlite3.connect('superstore.db')


df.to_sql('orders', conn, if_exists='replace', index=False)


9800

## 📈 Query 1: Monthly Order Volume

This query counts the total number of unique orders placed each month.  
It helps identify seasonality and order volume trends over time.

In [21]:
query1 = """
SELECT 
    strftime('%Y-%m', "Order Date") AS order_month,
    COUNT(DISTINCT "Order ID") AS total_orders
FROM orders
GROUP BY order_month
ORDER BY order_month;

"""
pd.read_sql_query(query1, conn)

,order_month,total_orders
0,2015-01,30
1,2015-02,28
2,2015-03,69
3,2015-04,63
4,2015-05,68
5,2015-06,64
6,2015-07,64
7,2015-08,70
8,2015-09,129
9,2015-10,78


## 👑 Query 2: Top 5 Customers by Sales

This query ranks customers by total revenue generated.  
Such analysis helps target high-value customers for loyalty programs or retention efforts.

In [22]:
query2 = """
SELECT 
    "Customer Name",
    ROUND(SUM(Sales), 2) AS total_sales
FROM orders
GROUP BY "Customer Name"
ORDER BY total_sales DESC
LIMIT 5;

"""
pd.read_sql_query(query2, conn)

,Customer Name,total_sales
0,Sean Miller,25043.05
1,Tamara Chand,19052.22
2,Raymond Buch,15117.34
3,Tom Ashbrook,14595.62
4,Adrian Barton,14473.57


## 📦 Query 3: Top 5 Products by Total Sales

This shows which products contributed the most to overall revenue — useful for inventory management and product promotions.

In [13]:
query3 = """
SELECT 
    "Product Name",
    ROUND(SUM(Sales), 2) AS total_sales
FROM orders
GROUP BY "Product Name"
ORDER BY total_sales DESC
LIMIT 5;

"""
pd.read_sql_query(query3, conn)

,Product Name,total_sales
0,Canon imageCLASS 2200 Advanced Copier,61599.82
1,Fellowes PB500 Electric Punch Plastic Comb Bin...,27453.38
2,Cisco TelePresence System EX90 Videoconferenci...,22638.48
3,HON 5400 Series Task Chairs for Big and Tall,21870.58
4,GBC DocuBind TL300 Electric Binding System,19823.48


## ⏱️ Query 4: Average Delivery Time (in Days)

Using SQLite's `JULIANDAY()` function, we calculate the average delivery time between order and shipment.  
This helps measure operational efficiency.

In [23]:
query4 = """
SELECT 
    ROUND(AVG(JULIANDAY("Ship Date") - JULIANDAY("Order Date")), 2) AS avg_delivery_days
FROM orders;

"""
pd.read_sql_query(query4, conn)

,avg_delivery_days
0,3.96


## 🌍 Query 5: Orders by Region

We group orders by geographical regions to identify high-performing or underperforming zones.

In [24]:
query5 = """
SELECT 
    Region,
    COUNT(DISTINCT "Order ID") AS total_orders
FROM orders
GROUP BY Region;
"""
pd.read_sql_query(query5,conn)

,Region,total_orders
0,Central,1156
1,East,1369
2,South,810
3,West,1587


## 🧩 Query 6: Sales by Product Category

This aggregates sales based on product categories to understand which broad product lines are most profitable.

In [25]:
query6 = """
SELECT 
    Category,
    ROUND(SUM(Sales), 2) AS total_sales
FROM orders
GROUP BY Category
ORDER BY total_sales DESC;
"""
pd.read_sql_query(query6,conn)

,Category,total_sales
0,Technology,827455.87
1,Furniture,728658.58
2,Office Supplies,705422.33


## ✅ Conclusion

This project highlights how SQL can be used for quick, powerful insights from structured business data.

We performed:
- Time-based sales trend analysis
- Customer and product segmentation
- Operational efficiency measurement
- Regional and category-level aggregation

This project demonstrates end-to-end data analysis using Python, SQL, and business logic — exactly the kind of work expected from Data Analysts and Business Analysts in real companies.
